In [1]:
import pandas as pd
from pyvi import ViTokenizer
import re
import string
import codecs
import time

In [2]:
#Từ điển tích cực, tiêu cực, phủ định

path_nag = 'data3/nag.txt'
path_pos = 'data3/pos.txt'
path_not = 'data3/not.txt'

In [3]:
with codecs.open(path_nag, 'r', encoding='UTF-8') as f:
    nag = f.readlines()
nag_list = [n.replace('\n', '') for n in nag]

with codecs.open(path_pos, 'r', encoding='UTF-8') as f:
    pos = f.readlines()
pos_list = [n.replace('\n', '') for n in pos]

with codecs.open(path_not, 'r', encoding='UTF-8') as f:
    not_ = f.readlines()
not_list = [n.replace('\n', '') for n in not_]

In [4]:
VN_CHARS_LOWER = u'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđð'
VN_CHARS_UPPER = u'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸÐĐ'
VN_CHARS = VN_CHARS_LOWER + VN_CHARS_UPPER

# Hàm này dùng để remove dấu \ / ? ~ . trong câu

def no_marks(s):
    __INTAB = [ch for ch in VN_CHARS]
    __OUTTAB = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"*2
    __OUTTAB += "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"*2
    __r = re.compile("|".join(__INTAB))
    __replaces_dict = dict(zip(__INTAB, __OUTTAB))
    result = __r.sub(lambda m: __replaces_dict[m.group(0)], s)
    return result

In [5]:
def normalize_text(text):

    #Remove các ký tự kéo dài: vd: đẹppppppp
    text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)

    # Chuyển thành chữ thường
    text = text.lower()

    #Chuẩn hóa tiếng Việt, xử lý emoj, chuẩn hóa tiếng Anh, thuật ngữ
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
        "👹": "nagative", "👻": "positive", "💃": "positive",'🤙': ' positive ', '👍': ' positive ',
        "💄": "positive", "💎": "positive", "💩": "positive","😕": "nagative", "😱": "nagative", "😸": "positive",
        "😾": "nagative", "🚫": "nagative",  "🤬": "nagative","🧚": "positive", "🧡": "positive",'🐶':' positive ',
        '👎': ' nagative ', '😣': ' nagative ','✨': ' positive ', '❣': ' positive ','☀': ' positive ',
        '♥': ' positive ', '🤩': ' positive ', 'like': ' positive ', '💌': ' positive ',
        '🤣': ' positive ', '🖤': ' positive ', '🤤': ' positive ', ':(': ' nagative ', '😢': ' nagative ',
        '❤': ' positive ', '😍': ' positive ', '😘': ' positive ', '😪': ' nagative ', '😊': ' positive ',
        '?': ' ? ', '😁': ' positive ', '💖': ' positive ', '😟': ' nagative ', '😭': ' nagative ',
        '💯': ' positive ', '💗': ' positive ', '♡': ' positive ', '💜': ' positive ', '🤗': ' positive ',
        '^^': ' positive ', '😨': ' nagative ', '☺': ' positive ', '💋': ' positive ', '👌': ' positive ',
        '😖': ' nagative ', '😀': ' positive ', ':((': ' nagative ', '😡': ' nagative ', '😠': ' nagative ',
        '😒': ' nagative ', '🙂': ' positive ', '😏': ' nagative ', '😝': ' positive ', '😄': ' positive ',
        '😙': ' positive ', '😤': ' nagative ', '😎': ' positive ', '😆': ' positive ', '💚': ' positive ',
        '✌': ' positive ', '💕': ' positive ', '😞': ' nagative ', '😓': ' nagative ', '️🆗️': ' positive ',
        '😉': ' positive ', '😂': ' positive ', ':v': '  positive ', '=))': '  positive ', '😋': ' positive ',
        '💓': ' positive ', '😐': ' nagative ', ':3': ' positive ', '😫': ' nagative ', '😥': ' nagative ',
        '😃': ' positive ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' positive ', '🤝': ' positive ', '🎈': ' positive ',
        '😗': ' positive ', '🤔': ' nagative ', '😑': ' nagative ', '🔥': ' nagative ', '🙏': ' nagative ',
        '🆗': ' positive ', '😻': ' positive ', '💙': ' positive ', '💟': ' positive ',
        '😚': ' positive ', '❌': ' nagative ', '👏': ' positive ', ';)': ' positive ', '<3': ' positive ',
        '🌝': ' positive ',  '🌷': ' positive ', '🌸': ' positive ', '🌺': ' positive ',
        '🌼': ' positive ', '🍓': ' positive ', '🐅': ' positive ', '🐾': ' positive ', '👉': ' positive ',
        '💐': ' positive ', '💞': ' positive ', '💥': ' positive ', '💪': ' positive ',
        '💰': ' positive ',  '😇': ' positive ', '😛': ' positive ', '😜': ' positive ',
        '🙃': ' positive ', '🤑': ' positive ', '🤪': ' positive ','☹': ' nagative ',  '💀': ' nagative ',
        '😔': ' nagative ', '😧': ' nagative ', '😩': ' nagative ', '😰': ' nagative ', '😳': ' nagative ',
        '😵': ' nagative ', '😶': ' nagative ', '🙁': ' nagative ',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' positive ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
        ' lol ': ' nagative ',' cc ': ' nagative ','cute': u' dễ thương ','huhu': ' nagative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        #dưới 3* quy về 1*, trên 3* quy về 5*
        '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
        'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
        '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ',}

    for k, v in replace_list.items():
        text = text.replace(k, v)

    # chuyen punctuation thành space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    text = ViTokenizer.tokenize(text)
    texts = text.split()
    len_text = len(texts)

    texts = [t.replace('_', ' ') for t in texts]
    for i in range(len_text):
        cp_text = texts[i]
        if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
            numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1

            for j in range(numb_word):
                if texts[i + j + 1] in pos_list:
                    texts[i] = 'notpos'
                    texts[i + j + 1] = ''

                if texts[i + j + 1] in nag_list:
                    texts[i] = 'notnag'
                    texts[i + j + 1] = ''
        else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp positive)
            if cp_text in pos_list:
                texts.append('positive')
            elif cp_text in nag_list:
                texts.append('nagative')

    text = u' '.join(texts)

    #remove nốt những ký tự thừa thãi
    text = text.replace(u'"', u' ')
    text = text.replace(u'️', u'')
    text = text.replace('🏻','')
    return text

In [6]:
class DataSource(object):

    def _load_raw_data(self, filename, is_train=True):

        a = []
        b = []

        regex = 'train_'
        if not is_train:
            regex = 'test_'

        with open(filename, 'r', encoding="utf8") as file:
            for line in file:
                if regex in line:
                    b.append(a)
                    a = [line]
                elif line != '\n':
                    a.append(line)
        b.append(a)

        return b[1:]

    def _create_row(self, sample, is_train=True):

        d = {}
        d['id'] = sample[0].replace('\n', '')
        review = ""

        if is_train:
            for clause in sample[1:-1]:
                review += clause.replace('\n', ' ')
                review = review.replace('.', ' ')

            d['label'] = int(sample[-1].replace('\n', ' '))
        else:
            for clause in sample[1:]:
                review += clause.replace('\n', ' ')
                review = review.replace('.', ' ')


        d['review'] = review

        return d

    def load_data(self, filename, is_train=True):

        raw_data = self._load_raw_data(filename, is_train)
        lst = []

        for row in raw_data:
            lst.append(self._create_row(row, is_train))

        return lst

    def transform_to_dataset(self, x_set,y_set):
        X, y = [], []
        for document, topic in zip(list(x_set), list(y_set)):
            document = normalize_text(document)
            X.append(document.strip())
            y.append(topic)
            #Augmentation bằng cách remove dấu tiếng Việt
            #X.append(no_marks(document))
           # y.append(topic)
        return X, y

In [7]:
# Loading file training

ds = DataSource()
train_data = pd.DataFrame(ds.load_data('data3/train.crash'))


In [8]:
#Thêm mẫu bằng cách lấy trong từ điển Sentiment (nag/pos)

for index,row in enumerate(nag_list):
    new_data.append(['pos'+str(index),'0',row])
for index,row in enumerate(nag_list):
    new_data.append(['nag'+str(index),'1',row])

In [9]:
# Loading file testing
new_data = []
new_data = pd.DataFrame(new_data,columns=list(['id','label','review']))
train_data.append(new_data)

#test_data = pd.DataFrame(ds.load_data('data3/test.crash', is_train=False))

,id,label,review
0,train_000000,0,"""Dung dc sp tot cam on shop Đóng gói sản phẩm ..."
1,train_000001,0,""" Chất lượng sản phẩm tuyệt vời Son mịn nhưn..."
2,train_000002,0,""" Chất lượng sản phẩm tuyệt vời nhưng k có hộp..."
3,train_000003,1,""":(( Mình hơi thất vọng 1 chút vì mình đã kỳ v..."
4,train_000004,1,"""Lần trước mình mua áo gió màu hồng rất ok mà ..."
...,...,...,...
16068,train_016082,1,"""Chẳng biết là Shop có biết đọc hay không mua ..."
16069,train_016083,1,"""Cuốn này mỏng Đọc một buổi sáng là hết Thú ..."
16070,train_016084,0,"""Mang êm chân Đẹp """
16071,train_016085,1,"""Tôi đã nhận đc hàng Sau đây là vài lời muốn n..."


In [10]:
X_train = train_data.review
y_train = train_data.label

In [11]:
#print(X_train[1],y_train[1])

Phần này thực hiện các module augmentation

In [12]:
# Load danh sách các từ stop word
stoplist = []

with open("./data/vnstopword.txt", encoding="utf-8") as f :
  text = f.read()
  for word in text.split('\n') :  #Tách ra mỗi dòng là 1 từ stopword riêng lẻ
      stoplist.append(word)
  f.close()

In [13]:
from gensim.models import KeyedVectors, Word2Vec

# Load mô hình trained word2vec để embedding
word2vec_model_path = ("./data/w2v.bin")

#w2v = Word2Vec.load(word2vec_model_path)
w2v = KeyedVectors.load_word2vec_format(word2vec_model_path,binary = True)

In [14]:
from nltk import word_tokenize
from pyvi import ViTokenizer, ViPosTagger
import random

#====================================================================================================
# Hàm lấy từ đồng nghĩa trong file vietnamsyn.txt
#synonyms_lexicon = get_synonyms('./vietnamsyn.txt')

def get_synonyms(path):
    synonyms_lexicon = {}
    text_entries = [l.strip() for l in open(path, encoding="utf8").readlines()]
    for e in text_entries:
        e = e.split('\t')
        k = e[0]
        v = e[1:len(e)]
        synonyms_lexicon[k] = v
    return synonyms_lexicon

#====================================================================================================
# Hàm hoán đổi ngẫu nhiên vị trí các từ trong câu. Số lượng câu mới tạo ra là n

def Random_Swap(sentence,n):
    new_sentences = []
    words = sentence.split()
    for i in range(n):
        random.shuffle(words)
        new_sentences.append(' '.join(words))
    return new_sentences

#====================================================================================================
# Hàm xoá từ trong câu nếu từ đó là: Động từ (V), giới từ (C), số (M), dấu câu (F)

def Random_Deletion(sentence):
    
    new_sentence = []
    tagged_word = ViPosTagger.postagging(sentence)
    
    # Chia POS thành 2 list cho dễ thực hiện
    word = tagged_word[0]
    tag = tagged_word[1]

    edited_sentence = [i for i,j in zip(word,tag) if j != 'P' and j != 'V' and j != 'C' and j != 'F' and j != 'M']
    edited_sentence = ' '.join(edited_sentence)
    new_sentence.append(edited_sentence)
    return new_sentence

#====================================================================================================
# Hàm thay thế từ đồng nghĩa

def syn_rep(sentence, synonyms_lexicon):
    keys = synonyms_lexicon.keys()
    words = sentence.split()
    n_sentence = sentence
    for w in words:
        if w not in stoplist:
            if w in keys:
                n_sentence = n_sentence.replace(w, synonyms_lexicon[w][0])  # Thay đổi từ đồng nghĩa ở cột kế tiếp
    return n_sentence

#===================================================================================

def Synonym_Replacement(sentence):
    #Get synonyms word from this file
    synonyms_lexicon = get_synonyms('./data/vietnamsyn.txt')
    
    new_sentence = []
    
    sen_replaced = syn_rep(sentence, synonyms_lexicon)
    new_sentence.append(sen_replaced)
    
    return new_sentence

#====================================================================================================
# Hàm chèn từ vào câu
def Insert(sentence, synonyms_lexicon):
    keys = synonyms_lexicon.keys()
    words = sentence.split()
    n_sentence = sentence
    for w in words:
        if w not in stoplist:
            if w in keys:
                n_sentence = n_sentence + ' ' + synonyms_lexicon[w][0] # Chèn từ đồng nghĩa vào cuối câu.
    return n_sentence

#===================================================================================

def Random_Insert(sentence):
    #Get synonyms word from this file
    synonyms_lexicon = get_synonyms('./data/vietnamsyn.txt')
    
    new_sentence = []
    
    sen_inserted = Insert(sentence, synonyms_lexicon)
    new_sentence.append(sen_inserted)
    
    return new_sentence

#====================================================================================================
# Finding out a similarity word from word embedding space

def Similarity(word):
    # Lấy ra 1 similarity word đầu tiên với score lớn nhất
    word_similarity = w2v.most_similar(word,topn=1)
    
    # Vòng lặp này để trả về word đầu tiên, với similarity score là lớn nhất
    for x in word_similarity:
        word = ''.join(x[0]) # Lệnh này lấy ra chữ, bỏ qua score
    return word   

#===================================================================================
# Repalcement similitary word from word vector embedding space

def Word_Replacement(sentence):
    words = sentence.split()
    replaced_sentence = []
    new_sentence = ''
    for word in words:
        if word not in w2v:
            new_sentence = new_sentence + ' ' + word
        else: 
            if word in stoplist:
                new_sentence = new_sentence + ' ' + word
            else:
                new_word = Similarity(word)
                new_sentence = new_sentence + ' ' + new_word
                
    replaced_sentence.append(new_sentence.strip())
    return replaced_sentence

In [15]:
# Đoạn code này là đọc tất cả các comment và label. Thực hiện agumentation.
# Sau đó lưu vào file .csv
import csv   
header=['comment','label']
with open(r'data3.csv', 'a', encoding='utf8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(0,len(X_train)):
        comment = str(X_train[i])
        label = y_train[i]
        data = [comment,label]
        writer.writerow(data)
        
        # Thực hiện swapping
        data= [Random_Swap(comment,1),y_train[i]]
        writer.writerow(data)
    
        # Thực hiện thay thế từ đồng nghĩa
        #data= [Synonym_Replacement(comment),y_train[i]]
        #writer.writerow(data)
        
        # Thực hiện việc xoá ngẫu nhiên
        data= [Random_Deletion(comment),y_train[i]]
        writer.writerow(data)
        
        # Thực hiện việc chèn từ đồng nghĩa cuối câu
        data= [Random_Insert(comment),y_train[i]]
        writer.writerow(data)
        
        # Thực hiện việc thay thế từ gần nghĩa ()
        #data= [Word_Replacement(comment),y_train[i]]
        #writer.writerow(data)

In [16]:
import pandas as pd
a = pd.read_csv('data3.csv', encoding='utf8')

In [27]:
a.shape

(64301, 2)